In [87]:
'''
Reference:
https://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/
'''

'\nReference:\nhttps://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/\n'

In [88]:
import tweepy
from tweepy import OAuthHandler

In [89]:
import config as cf 
cf.consumer_key
cf.consumer_secret
cf.access_token
cf.access_secret

'ij4sptlcUgeBaNZuHfcha7ZWZHMVnWDrnwg7ZFO8FjKMo'

In [90]:
auth = OAuthHandler(consumer_key, consumer_secret) #先用鑰匙開門
auth.set_access_token(access_token, access_secret) #找到房號進去

api = tweepy.API(auth)

In [91]:
#read our own timeline
for status in tweepy.Cursor(api.home_timeline).items(10): 
    print(status.text) #number above is how many tweet we read
    

Oooooh, so this is why Daybreak (former Sony Online Entertainment) suddenly erased all mentions of Columbus Nova ow… https://t.co/crg1RMBAZ8
Any image transfer to txt https://t.co/xqmBTLCAEf
Help with a storyboard I'm working on https://t.co/UyV3Hr2Bw8
RT @videodante: One of my favorite parts of Batletech is its character diversity, and I barely touched on it in my review. Let's break down…
Ten simple rules to use statistics effectively https://t.co/VpwE71GWWd
CRUD App using Vue.js and Django

☞ https://t.co/GEecawWqFJ

#Django #Vuejs https://t.co/BvwYxGGtAQ
RT @hsbt: 今年も RailsGirls 卒業生向けに RubyKaigi 2018 参加への交通費・宿泊費支援をやることにしました! 対象の方は奮ってご応募ください〜。 https://t.co/m61ZElLvxS #railsgirls #rubykaigi
The documentary heritage chosen covers #Taiwanese society, politics, religion, education, indigenous history, and w… https://t.co/yTTCQU6zfM
The register also includes documentary #heritage on the Republic of #Formosa. Although the time between its 1895 fo… https://t.co/5uOZWmJZRg
What is Python? 

In [92]:
#print the same in JSON type(not string, but dictionary)
for status in tweepy.Cursor(api.home_timeline).items(10):
    print(status._json) #interested_things.type

{u'contributors': None, u'truncated': True, u'text': u'Oooooh, so this is why Daybreak (former Sony Online Entertainment) suddenly erased all mentions of Columbus Nova ow\u2026 https://t.co/crg1RMBAZ8', u'is_quote_status': True, u'in_reply_to_status_id': None, u'id': 994490883289567236, u'favorite_count': 0, u'source': u'<a href="http://tapbots.com/tweetbot" rel="nofollow">Tweetbot for i\u039fS</a>', u'quoted_status_id': 994283426114080768, u'retweeted': False, u'coordinates': None, u'quoted_status': {u'contributors': None, u'truncated': True, u'text': u"A senior official inside the drug giant Novartis told NBC News that Michael Cohen reached out shortly after Trump's\u2026 https://t.co/yo3lk6MlUW", u'is_quote_status': False, u'in_reply_to_status_id': None, u'id': 994283426114080768, u'favorite_count': 1319, u'source': u'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', u'retweeted': False, u'coordinates': None, u'entities': {u'symbols': [], u'user_me

In [93]:
#Custom function to process of store:
import json #in 2, this step is indispensable
def process_or_store(tweet):
    print(json.dumps(tweet)) #turn the tweet into a JSON type

In [94]:
for friend in tweepy.Cursor(api.friends).items(5):
    process_or_store(friend._json)

{"follow_request_sent": false, "has_extended_profile": false, "profile_use_background_image": false, "live_following": false, "default_profile_image": false, "id": 738267072250028036, "profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme1/bg.png", "translator_type": "none", "verified": true, "blocked_by": false, "profile_text_color": "000000", "muting": false, "profile_image_url_https": "https://pbs.twimg.com/profile_images/948008652425674752/ikfMWmhI_normal.jpg", "profile_sidebar_fill_color": "000000", "entities": {"url": {"urls": [{"url": "https://t.co/ekfaoN8fRL", "indices": [0, 23], "expanded_url": "http://english.moc.gov.tw/", "display_url": "english.moc.gov.tw"}]}, "description": {"urls": []}}, "followers_count": 7233, "profile_sidebar_border_color": "000000", "id_str": "738267072250028036", "profile_background_color": "000000", "listed_count": 106, "status": {"contributors": null, "truncated": true, "text": "The documentary heritage chosen covers #Taiw

In [95]:
#a list of all our tweets
for tweet in tweepy.Cursor(api.user_timeline).items(5):
    print(tweet._json)

# Streaming

In [96]:
from tweepy import Stream
from tweepy.streaming import StreamListener

In [13]:
class MyListener(StreamListener):
    
    def on_data(self, data):
        try:
            with open('Hawaii.json', 'a') as f: #opens the file for appending, 'python.json' for file name
                f.write(data)
                return True
        except BaseException as e:
            print('Error on_data: %s' % str(e))
        return True
    
    def on_error(self, status):
        print(status)
        return True
    
twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['Hawaii'])

KeyboardInterrupt: 

# The Anatomy of a Tweet

In [109]:
import json

with open('Hawaii.json', 'r') as f: #structure of a tweet
    line = f.readline() #read only the first tweet
    tweet = json.loads(line) #load it as Python dict
    print(json.dumps(tweet, indent=4))


#json.loads: load json as dict
#json.dumps: load dict as json

{
    "quote_count": 0, 
    "contributors": null, 
    "truncated": false, 
    "text": "RT @MBVD: An offering, known in Hawaiian as a ho`okupu, has been left for Pele, the goddess of fire and lava, on Hookupu Street in front of\u2026", 
    "is_quote_status": false, 
    "in_reply_to_status_id": null, 
    "reply_count": 0, 
    "id": 993846832995454982, 
    "favorite_count": 0, 
    "entities": {
        "user_mentions": [
            {
                "id": 106010958, 
                "indices": [
                    3, 
                    8
                ], 
                "id_str": "106010958", 
                "screen_name": "MBVD", 
                "name": "Michelle Broder Van Dyke"
            }
        ], 
        "symbols": [], 
        "hashtags": [], 
        "urls": []
    }, 
    "retweeted": false, 
    "coordinates": null, 
    "timestamp_ms": "1525786524424", 
    "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>", 

None


In [99]:
from nltk.tokenize import word_tokenize
print(word_tokenize(tweet['text'])) #unicode

[u'RT', u'@', u'MBVD', u':', u'An', u'offering', u',', u'known', u'in', u'Hawaiian', u'as', u'a', u'ho`okupu', u',', u'has', u'been', u'left', u'for', u'Pele', u',', u'the', u'goddess', u'of', u'fire', u'and', u'lava', u',', u'on', u'Hookupu', u'Street', u'in', u'front', u'of\u2026']


In [100]:
#tweet usually in unicode -> transform

tweet = tweet['text'].encode('ascii','ignore')
print(word_tokenize(tweet))

['RT', '@', 'MBVD', ':', 'An', 'offering', ',', 'known', 'in', 'Hawaiian', 'as', 'a', 'ho`okupu', ',', 'has', 'been', 'left', 'for', 'Pele', ',', 'the', 'goddess', 'of', 'fire', 'and', 'lava', ',', 'on', 'Hookupu', 'Street', 'in', 'front', 'of']


In [101]:
#the tweet be divided into too seperate
import re

emoticons_str = r'''
    (?:
        [:=;] #eye
        [oO\-] #nose
        [D\)\]\)\]/\\OpP] #mouth
    )'''

regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

#re.compile(pattern) comile the pattern of re.
#ref: http://zwindr.blogspot.jp/2016/01/python-regular-expression.html
tokens_re = re.compile(r'(' + '|'.join(regex_str) + ')', re.X |re.I)
emoticon_re = re.compile(r'^' + emoticons_str + '$', re.X | re.I)
#I -> ignorance: take the lowercase and uppercase as same
#X -> Verbose: make a re. more readable, ignore the '#' explantion


def tokenize(s):
    return tokens_re.findall(s)

def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower for token in tokens]
    return tokens

#re.search:Scan through string looking for the first 
#      location where the regular expression pattern 
#      produces a match

print(preprocess(tweet))

['RT', '@MBVD', ':', 'An', 'offering', ',', 'known', 'in', 'Hawaiian', 'as', 'a', 'ho', '`', 'okupu', ',', 'has', 'been', 'left', 'for', 'Pele', ',', 'the', 'goddess', 'of', 'fire', 'and', 'lava', ',', 'on', 'Hookupu', 'Street', 'in', 'front', 'of']


In [71]:
from textblob import TextBlob

#read all tweet
with open('Hawaii.json') as f:
    for line in f:
        tweet = json.loads(line)
        text = tweet['text'].encode('ascii','ignore')
        blob = TextBlob(text)
        dic = {'Token': preprocess(text),
               'Sentiment': blob.sentiment.polarity}
        print(dic)
        

{'Token': ['RT', '@MBVD', ':', 'An', 'offering', ',', 'known', 'in', 'Hawaiian', 'as', 'a', 'ho', '`', 'okupu', ',', 'has', 'been', 'left', 'for', 'Pele', ',', 'the', 'goddess', 'of', 'fire', 'and', 'lava', ',', 'on', 'Hookupu', 'Street', 'in', 'front', 'of'], 'Sentiment': 0.0}
{'Token': ['RT', '@webcamsdemexico', ':', 'La', 'lava', 'sigue', 'su', 'marcha', 'a', 'travs', 'de', '#Hawaii', ',', 'destruyendo', 'todo', 'a', 'su', 'paso', '.', 'Video', ':', '@CNNweather', '#VolcanKilauea', 'https://t'], 'Sentiment': 0.0}
{'Token': ['RT', '@bclemms', ':', 'No', 'words', 'can', 'describe', 'this', 'scene', '.', '.', '.', 'View', 'our', 'ongoing', 'video', 'coverage', 'of', 'the', 'situation', 'in', '#Hawaii', 'Hawaii', 'Kilauea', '#Volcano', '2018'], 'Sentiment': 0.0}
{'Token': ['Uh', 'I', 'just', 'saw', 'the', 'Avengers', 'and', 'prettttty', 'sure', 'Wakanda', 'has', 'some', 'stuff', '.', 'https://t.co/SzJwvyBJc0'], 'Sentiment': 0.5}
{'Token': ['RT', '@ABC', ':', 'This', 'was', 'the', 'scorc

{'Token': ['RT', '@NOAASatellites', ':', 'On', 'the', 'afternoon', 'of', 'April', '30,', '2018', 'an', '#earthquake', '"', 'swarm', '"', '(', 'up', 'to', '10', 'per', 'hour', ')', 'began', 'to', 'occur', 'in', '#Hawaii', 'with', 'volcanic', 'e'], 'Sentiment': 0.0}
{'Token': ['I', 'wonder', 'if', 'their', 'premiums', 'will', 'go', 'up', '?', '#hawaii', '#volcano', 'https://t.co/Kjjif0hRwz'], 'Sentiment': 0.0}
{'Token': ['RT', '@RogenSandy', ':', 'Hawaii', 'is', 'the', 'only', 'place', 'that', 'dudes', 'can', 'all', 'wear', 'the', 'same', 'shirt', 'and', 'its', 'cool', 'for', 'them', '.', 'I', 'would', 'always', 'think', 'it', 'was', 'coo'], 'Sentiment': 0.11666666666666665}
{'Token': ['RT', '@newscientist', ':', 'Hawaii', 'volcano', 'is', 'causing', 'havoc', 'and', 'will', 'spew', 'lava', 'for', 'days', 'https://t.co/Mr5K6hL1zW', 'https://t.co/virecclcN9'], 'Sentiment': 0.0}
{'Token': ['RT', '@TIME', ':', 'Hawaii', 'hit', 'with', '1,301', 'earthquakes', ':', 'The', 'latest', 'on', 'the'

{'Token': ['Metal', 'Fence', "Can't", 'Stop', 'Lava', 'from', "Hawaii's", 'Kilauea', 'Volcano', 'https://t.co/vsEm7LqybY'], 'Sentiment': 0.0}
{'Token': ['Beltway', 'Insider', ':', 'Trump', 'Angers', 'Allies', 'https://t.co/IrhQRsKpis'], 'Sentiment': 0.0}
{'Token': ['RT', '@UKWeatherLive', ':', 'Lava', 'eruptions', 'is', '#Hawaii', 'over', 'the', 'past', 'few', 'days', '.', 'Looks', 'like', 'a', 'scene', 'from', 'a', 'movie', 'https://t.co/8i7fPeMoYe'], 'Sentiment': -0.225}
{'Token': ['RT', '@MillenPolitics', ':', 'Hawaii', 'is', 'erupting', '.', 'Where', 'is', 'our', 'President', '?', 'How', 'hard', 'is', 'it', 'to', 'tweet', 'words', 'of', 'support', 'to', 'those', 'in', 'harms', 'way', '?', 'We', 'need', 'to'], 'Sentiment': -0.2916666666666667}
{'Token': ['RT', '@vicenews', ':', 'Oozing', 'lava', 'has', 'destroyed', 'dozens', 'of', 'homes', 'on', 'the', 'island', 'of', 'Hawaii', 'over', 'the', 'past', 'few', 'days', 'https://t.co/jpNtLePZk5'], 'Sentiment': -0.225}
{'Token': ['RT', '@

{'Token': ['RT', '@GustavoVela71', ':', 'Lava', '.', 'Hawaii', 'https://t.co/CmJmU0hZ0i'], 'Sentiment': 0.0}
{'Token': ['RT', '@9Kamol', ':', 'Secret', 'Hawaii', 'Citrus', 'Clear', 'Gel'], 'Sentiment': -0.15}
{'Token': ['RT', '@MillenPolitics', ':', 'Hawaii', 'is', 'erupting', '.', 'Where', 'is', 'our', 'President', '?', 'How', 'hard', 'is', 'it', 'to', 'tweet', 'words', 'of', 'support', 'to', 'those', 'in', 'harms', 'way', '?', 'We', 'need', 'to'], 'Sentiment': -0.2916666666666667}
{'Token': ['RT', '@ReporteYa', ':', '#8May', 'Secuencia', 'en', 'cmara', 'rpida', 'del', 'avance', 'de', 'la', 'lava', 'en', '#Hawaii', 'Es', 'increble', 'la', 'capacidad', 'de', 'destruccin', 'que', 'tiene', 'en', 'ocas'], 'Sentiment': 0.0}
{'Token': ['Experience', 'the', 'World', 'Famous', 'Manta', 'Ray', 'Night', 'Snorkel', 'in', 'Kona', ',', 'Hawaii', 'with', 'Jacks', 'Diving', 'Locker', ':', 'https://t.co/LGMBnc6AuQ'], 'Sentiment': 0.5}
{'Token': ['RT', '@NBC12', ':', 'Lava', 'from', 'the', 'Kilauea', 

In [128]:
with open('Happy.json') as f:
    for line in f:
        tweet = json.loads(line)
        #text = tweet['text'].encode('ascii','ignore')
        print(tweet['text'])
        
        
        #if re.match(r'^' + emoticons_str + '$',) != None:
        #    emo_dec.append(text)


This thread is everything and more. I’m so so happy with my life right now, it’s even scary. Don’t let them drain u… https://t.co/zqsP4FV3gm
RT @azwant: Ok la babi, nama sama dengan Azwan Ali ni lagi sayu. https://t.co/uz3wgphVPj
@TheQueenBLifee Girl, I love your positive outlook. Sounds just like me and makes me so happy there are other empow… https://t.co/4btrC1WJ2Z
This video still makes me so happy. https://t.co/rM7n6x5cYc
RT @writtenarts: Be with someone that makes you happy.
After so many weeks of work and sleepless nights, I’m super happy to… https://t.co/1jFtrwnGm3
RT @touchdalight: i can go from happy to sad in the space of 0.2 seconds i love being talented
@romero Happy birthday Wolf3D!!!
RT @LaughingLatina: Erotic #LapDance &amp; Happy Ending !#AssWorship #StripTease #RT @iWantClips https://t.co/SG9Ihlw85a https://t.co/MnkZEXNPCX
ka igat kay please tell me kay I swear you're free to go. Why would I stop you man if you're not happy anymore diba… https://t.co/HB7qt2dVhf
RT @IP

RT @GoogleForEdu: To all the teacher-heroes who take students on field trips and come back in one piece, this next @DonorsChoose category i…
#HappyBirthdaySampoo
Happy Birthday To @sampoornesh from @tarak9999 https://t.co/MJ6mNAhT3p
@Mrs_Carm @MrThreadzilla @z_williamson @CarmenCrincoli @juliehubs @TashasEv @98codes Fixed.. We're all happy now 🤠 https://t.co/gASp7upHaF
RT @CreationNA: Happy people are:
1 early sleepers
2 early risers
3 optimistic
4 risk takers
5 generous
6 not arrogant
7 honest
8 healthy
9…
@enriqueiglesias Happy birthday Enrique!!
RT @Anonymousmind01: Rupees at 68 and Petrol at 80. Head Coach Gobi ji and Supporting coach Jutley ji would be quite happy with performance…
RT @ThaTrapGod: My son either happy or pissed there is no inbetween https://t.co/jgAvSiJMSK
なんかオリックス強くね？ https://t.co/8UWJiQaAXq
RT @LlFEOFSTUDENTS: date privately. be happy privately. thrive privately. take your L’s privately.
RT @Rico_nastyy: This shit really made my birthday 😂😂😂 TORCH ASFFFFFFF https

RT @itsme_nalini: @xo_Juliet_Hana @harish_msd_7 @uthra_ajay @Rajifeb @aamina_twits @sweetzsona @soul_xxo @Ajmeeroviya @poornaoviyaa @Oviyaa…
RT @gabperey: OH MY GOD??? PLEASE RETWEET OH MY I’M SO BROKE AND THIS WOULD MAKE ME SO HAPPY aSsdhfkcJFKFK https://t.co/52eP0Ry1po
RT @iamallencecilio: Happy Birthday @maymayentrata07 ❤ https://t.co/bP7WswH9ow
RT @killthenoise: Happy birthday to my brotha @Madsonik @BrianTylerMusic have a good one man! Thinking of all the years we’ve been making n…
RT @infantethoughts: bebe ko bebe ko lang.

happy pill ko happy pill ko lang.

so back off.
RT @NaeeRachelle: I’M ABOUT TO BREAK DOWN 😩💯 LIKE I BEEN THROUGH SO MUCH SHIT UNNECESSARYILY 💔😞 I JUST WANT TO BE HAPPY &amp; AT PEACE , I’M TI…
RT @ThembaRumbu: Happy to announce that @RealBlackCoffee met with the Boy and bought the Portrait for R5800. Please Share for others to see…
RT @yowngeemean: @softiehobie HAPPY BIRTHDAY BITCH I HOPE BIGHIT FEEDS YOU WITH CONCEPT PHOTOS AGAIN TODAY
RT @Montaka37262266: น่

SyntaxError: unexpected character after line continuation character (<ipython-input-129-2da7aaf8e170>, line 2)

In [139]:
with open('Cry.json') as f:
    for line in f:
        tweet = json.loads(line)
        text = tweet['text'].encode('ascii','ignore')
        blob = TextBlob(text)
        #dic = {'Token': preprocess(text),
              # 'Sentiment': blob.sentiment.polarity}
        print(blob.words)
    
#u"\U0001F602"

['RT', 'choi_bts2', 'I', 'feel', 'like', 'there', 'will', 'be', 'one', 'more', 'Concept', 'photo', 'tonight', 'I', 'think', 'Ill', 'be', 'late', 'from', 'work', 'but', 'if', 'the', 'Concept', 'photo', 'wi']
['RT', 'faiqyuhaizad', 'For', 'a', 'better', 'Malaysia', 'jangan', 'mengharapkan', 'perubahan', 'daripada', 'kerajaan', 'semata2', 'rakyat', 'kena', 'berubah', 'gak', 'Sampah', 'jangan', 'bu']
['RT', 'HafizKrs', 'im', 'gon', 'na', 'miss', 'this', 'guy', 'https', 't.co/QrKxzOXHWq']
['RT', 'Iam_KhunGuy', 'WannaOne1TheWorldin']
['RT', 'OliviaResists', 'Is', 'this', 'our', 'America', 'A', 'Navy', 'Veteran', 'of', 'color', 'who', 'had', 'words', 'over', 'accidentally', 'placing', 'his', 'drink', 'on', 'a', 'table', 'Beaten']
['madisongesiotto']
['RT', 'choi_bts2', 'I', 'feel', 'like', 'there', 'will', 'be', 'one', 'more', 'Concept', 'photo', 'tonight', 'I', 'think', 'Ill', 'be', 'late', 'from', 'work', 'but', 'if', 'the', 'Concept', 'photo', 'wi']
['mcflurrymatcha', 'oo', 'puta', 'pogi',

In [ ]:
#寫一個有全部emoji unicode的list去stream
#一個unicode對應符號的dataframe
#

#先用喜怒哀樂的unicode分別抓500則
#斷詞之後算wordcount，但有些不是英文？？？？